In [1]:
# pip install opencv-python

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [6]:
def extract_orb_features(image_path):
    # Load image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    
    # Initialize ORB detector
    orb = cv2.ORB_create()
    
    # Find keypoints and descriptors
    kp, des = orb.detectAndCompute(img, None)
    return kp, des

def compute_similarity_bf(des1, des2):
    if des1 is None or des2 is None:
        return 0.0
    
    # Initialize BFMatcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Apply RANSAC to find the best transformation
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Use RANSAC to find the best transformation matrix
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Calculate the number of inliers
    num_inliers = np.sum(mask)

    # Compute the similarity score based on the number of inliers
    similarity_score = num_inliers / len(matches)
    return similarity_score

def compute_similarity_knn_ratio(des1, des2, ratio_threshold=0.75):
    if des1 is None or des2 is None:
        return 0.0
    
    # Initialize BFMatcher with NORM_HAMMING
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)

    # Match descriptors with KNN
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_threshold * n.distance:
            good_matches.append(m)

    if not good_matches:
        return 0.0  # No matches, no similarity

    # Apply RANSAC to find the best transformation
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Use RANSAC to find the best transformation matrix
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Calculate the number of inliers
    num_inliers = np.sum(mask)

    # Compute the similarity score based on the number of inliers
    similarity_score = num_inliers / len(good_matches)
    return similarity_score

def compute_similarity_flann(des1, des2):
    if des1 is None or des2 is None:
        return 0.0
    
    # FLANN parameters
    index_params = dict(algorithm=6,  # FLANN_INDEX_LSH
                        table_number=6,  # 12
                        key_size=12,     # 20
                        multi_probe_level=1)  # 2
    search_params = dict(checks=50)  # or pass an empty dictionary

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Match descriptors with KNN
    matches = flann.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if not good_matches:
        return 0.0  # No matches, no similarity

    # Apply RANSAC to find the best transformation
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Use RANSAC to find the best transformation matrix
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Calculate the number of inliers
    num_inliers = np.sum(mask)

    # Compute the similarity score based on the number of inliers
    similarity_score = num_inliers / len(good_matches)
    return similarity_score

def compute_similarity_avg_distance(des1, des2):
    if des1 is None or des2 is None:
        return 0.0
    
    # Match descriptors manually
    matches = match_descriptors(des1, des2)
    
    # Calculate the average distance of matches
    avg_distance = sum(match[2] for match in matches) / len(matches)

    # The similarity score can be the inverse of the average distance
    similarity_score = 1 / (1 + avg_distance)

    return similarity_score

def match_descriptors(des1, des2):
    matches = []
    for i, d1 in enumerate(des1):
        distances = np.linalg.norm(des2 - d1, axis=1)
        min_idx = np.argmin(distances)
        matches.append((i, min_idx, distances[min_idx]))
    return matches

def ratio_test(matches, ratio_threshold=0.75):
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_threshold * n.distance:
            good_matches.append(m)
    return good_matches

# Example usage:
image1_path = 'nk_collection_meubels_cleaned/meubel_1.jpg'
image2_path = 'nk_collection_meubels_cleaned/meubel_2.jpg'

kp1, des1 = extract_orb_features(image1_path)
kp2, des2 = extract_orb_features(image2_path)

similarity_score_bf = compute_similarity_bf(des1, des2)
print(f"Similarity Score (Brute-Force Matcher): {similarity_score_bf}")

similarity_score_knn_ratio = compute_similarity_knn_ratio(des1, des2)
print(f"Similarity Score (KNN Matcher with Ratio Test): {similarity_score_knn_ratio}")

similarity_score_flann = compute_similarity_flann(des1, des2)
print(f"Similarity Score (FLANN-Based Matcher): {similarity_score_flann}")

similarity_score_avg_distance = compute_similarity_avg_distance(des1, des2)
print(f"Similarity Score (Custom Average Distance): {similarity_score_avg_distance}")


Similarity Score (Brute-Force Matcher): 0.5434782608695652
Similarity Score (KNN Matcher with Ratio Test): 0.6712328767123288
Similarity Score (FLANN-Based Matcher): 0.7590361445783133
Similarity Score (Custom Average Distance): 0.0016686844702891743
